<a href="https://colab.research.google.com/github/summiyak/NLP_Final_project/blob/main/Model_Output_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [ ]:
!pip install -r /content/drive/MyDrive/electra/fp-dataset-artifacts-main/requirements.txt

In [4]:
import json
json_dataset = []  # Initialize an empty list to store JSON objects
with open("/content/drive/MyDrive/electra/output/eval_predictions.jsonl", 'r') as f:
    for line in f:  # Read the file line by line
        try:
            data = json.loads(line)  # Parse each line as a JSON object
            json_dataset.append(data)  # Add the parsed object to the list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}, skipping line: {line}")

In [5]:
print(json_dataset[0])

{'premise': 'Two women are embracing while holding to go packages.', 'hypothesis': 'The sisters are hugging goodbye while holding to go packages after just eating lunch.', 'label': 1, 'predicted_scores': [-2.337942361831665, 4.425786972045898, -2.92610239982605], 'predicted_label': 1}


In [6]:
import pandas as pd
from datasets import Dataset

dataset = Dataset.from_list(json_dataset)

In [5]:
import numpy as np
import pandas as pd
from datasets import Dataset

dataset = Dataset.from_list(json_dataset)
# Example Data (Replace with actual data)
data = json_dataset

num_classes = 3  # Number of label classes



def token_overlap_ratio(premise, hypothesis):
    premise_tokens = set(premise.split())
    hypothesis_tokens = set(hypothesis.split())
    intersection = len(premise_tokens & hypothesis_tokens)
    return intersection / len(premise_tokens)

# Define Error Classes
error_classes = {
    "Negation": lambda x: any(word in x["premise"].split(" ") + x["hypothesis"].split(" ") for word in ["not", "no", "never", "without"]),
    "Coreference": lambda x: any(word in x["hypothesis"] for word in ["he", "she", "it", "they"]),
    "Rare Words": lambda x: any(word in x["premise"] for word in ["Higgs", "boson", "quantum"]),
    "Ambiguity": lambda x: any(word in x["hypothesis"] for word in ["something", "someone", "several"]),
    "Long Sentences": lambda x: len(x["premise"].split()) > 10 or len(x["hypothesis"].split()) > 10,
    "High Lexical Overlap": lambda x: token_overlap_ratio(x["premise"], x["hypothesis"]) > 0.7,
}



# Initialize Counters
error_counts = {cls: 0 for cls in error_classes}
total_counts = {cls: 0 for cls in error_classes}
error_examples = {cls: [] for cls in error_classes}

# Process Examples
for example in data:
    true_label = example['label']
    predicted_label = example['predicted_label']
    is_error = true_label != predicted_label  # Check if the prediction is incorrect

    for cls, rule in error_classes.items():
        if rule(example):  # Check if example matches the error class
            total_counts[cls] += 1
            if is_error:
                error_counts[cls] += 1
                error_examples[cls].append(example)

# Calculate Metrics
error_rates = {cls: (error_counts[cls] / total_counts[cls] * 100 if total_counts[cls] > 0 else 0) for cls in error_classes}
frequencies = {cls: (total_counts[cls] / len(data) * 100) for cls in error_classes}

# Create Summary Table
summary_table = pd.DataFrame({
    "Error Class": list(error_classes.keys()),
    "Frequency (%)": [frequencies[cls] for cls in error_classes],
    "Error Rate (%)": [error_rates[cls] for cls in error_classes],
    "Error Count": [error_counts[cls] for cls in error_classes],
    "Total Count": [total_counts[cls] for cls in error_classes],
})

# Display Results
print(summary_table)


            Error Class  Frequency (%)  Error Rate (%)  Error Count  \
0              Negation       1.574883       12.258065           19   
1           Coreference      64.661654       11.345066          722   
2            Rare Words       0.000000        0.000000            0   
3             Ambiguity       1.158301       21.052632           24   
4        Long Sentences      68.258484       10.583507          711   
5  High Lexical Overlap       3.881325        8.900524           34   

   Total Count  
0          155  
1         6364  
2            0  
3          114  
4         6718  
5          382  


In [6]:

# (Optional) Print Example Errors
for cls, examples in error_examples.items():
    print(f"\nExamples for {cls}:")
    if cls == "High Lexical Overlap":
      for example in examples:
          print(example)


Examples for Negation:

Examples for Coreference:

Examples for Rare Words:

Examples for Ambiguity:

Examples for Long Sentences:

Examples for High Lexical Overlap:
{'premise': 'A girl climbing a rock face.', 'hypothesis': 'A girl climbing a rock face outside.', 'label': 1, 'predicted_scores': [1.876288652420044, 1.4281811714172363, -4.8618245124816895], 'predicted_label': 0}
{'premise': 'A person dressed in red and black outside a cracked wall.', 'hypothesis': 'A person in red and black near a brick wall.', 'label': 1, 'predicted_scores': [-0.8360059261322021, -0.0061300769448280334, 0.8703426718711853], 'predicted_label': 2}
{'premise': 'Indian women playing musical chairs with white woman.', 'hypothesis': 'indian women playing musical chairs with white woman in out door', 'label': 0, 'predicted_scores': [-1.081540584564209, 2.3348538875579834, -1.775390863418579], 'predicted_label': 1}
{'premise': 'A man reading the paper at a cafe.', 'hypothesis': 'A man reading the paper while 

**Testing HANS Strategy**

Processing evaluation set

In [7]:
from datasets import load_dataset
dataset = load_dataset("snli")
train_dataset_a = dataset["train"]

data = train_dataset_a


import json
hans_eval = []  # Initialize an empty list to store JSON objects
with open("/content/drive/MyDrive/electra/output_analysis/heuristics_evaluation_set.jsonl", 'r') as f:
    for line in f:  # Read the file line by line
        try:
            hans = json.loads(line)  # Parse each line as a JSON object
            hans_eval.append(hans)  # Add the parsed object to the list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}, skipping line: {line}")





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [12]:
print(hans_eval[0])

{'heuristic': 'lexical_overlap', 'template': 'temp1', 'sentence1_binary_parse': '( ( The president ) ( ( advised ( the doctor ) ) . ) )', 'sentence2_parse': '(ROOT (S (NP (DT The) (NN doctor)) (VP (VBD advised) (NP (DT the) (NN president))) (. .)))', 'subcase': 'ln_subject/object_swap', 'sentence2_binary_parse': '( ( The doctor ) ( ( advised ( the president ) ) . ) )', 'sentence1': 'The president advised the doctor .', 'sentence2': 'The doctor advised the president .', 'sentence1_parse': '(ROOT (S (NP (DT The) (NN president)) (VP (VBD advised) (NP (DT the) (NN doctor))) (. .)))', 'gold_label': 'non-entailment', 'pairID': 'ex0', 'label': 1}


In [ ]:

label_map_hans = {"entailment": 0, "non-entailment": 1}
for i in hans_eval:
  i["label"] = label_map_hans[i["gold_label"]]
print(hans_eval[0])


In [13]:



dataset_new = {}
print(len(hans_eval))
with open('/content/drive/MyDrive/electra/output_analysis/eval_hans_lexical_overlap.json', 'w') as f:
  for i in range(len(hans_eval)):
     if hans_eval[i]['heuristic'] == 'lexical_overlap' :
        if hans_eval[i]['label'] == -1 :
          label = 1
        else :
          label = hans_eval[i]['label']
        dataset_new.update( {
            'premise' : hans_eval[i]['sentence1']
            ,'hypothesis' : hans_eval[i]['sentence2']
            , 'label' : label
            })

        json.dump(dataset_new, f)
        f.write('\n')


30000


Pretrained model testing on HANS Evaluation set

In [4]:
!python3 /content/drive/MyDrive/electra/fp-dataset-artifacts-main/run.py --do_eval --task nli --dataset /content/drive/MyDrive/electra/output_analysis/HANS/eval_hans_lexical_overlap.json --model /content/drive/MyDrive/electra/trained_model/ --output_dir /content/drive/MyDrive/electra/output_analysis/HANS/output/

2024-11-28 01:24:17.744835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 01:24:17.777433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 01:24:17.787388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 01:24:17.810644: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 01:24:19.557102: W tensorflow/comp

Preparing HANS train set

In [7]:
from datasets import load_dataset



import json
hans_eval = []  # Initialize an empty list to store JSON objects
with open("/content/drive/MyDrive/electra/output_analysis/HANS/heuristics_train_set.jsonl", 'r') as f:
    for line in f:  # Read the file line by line
        try:
            hans = json.loads(line)  # Parse each line as a JSON object
            hans_eval.append(hans)  # Add the parsed object to the list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}, skipping line: {line}")


In [8]:

label_map_hans = {"entailment": 0, "non-entailment": 1}
for i in hans_eval:
  i["label"] = label_map_hans[i["gold_label"]]
print(hans_eval[0])


{'heuristic': 'lexical_overlap', 'template': 'temp1', 'sentence1_binary_parse': '( ( The doctors ) ( ( supported ( the scientist ) ) . ) )', 'sentence2_parse': '(ROOT (S (NP (DT The) (NN scientist)) (VP (VBD supported) (NP (DT the) (NNS doctors))) (. .)))', 'subcase': 'ln_subject/object_swap', 'sentence2_binary_parse': '( ( The scientist ) ( ( supported ( the doctors ) ) . ) )', 'sentence1': 'The doctors supported the scientist .', 'sentence2': 'The scientist supported the doctors .', 'sentence1_parse': '(ROOT (S (NP (DT The) (NNS doctors)) (VP (VBD supported) (NP (DT the) (NN scientist))) (. .)))', 'gold_label': 'non-entailment', 'pairID': 'ex0', 'label': 1}


In [9]:

dataset_new = {}
print(len(hans_eval))
with open('/content/drive/MyDrive/electra/output_analysis/HANS/train_hans_lexical_overlap.json', 'w') as f:
  for i in range(len(hans_eval)):
     if hans_eval[i]['heuristic'] == 'lexical_overlap' :
        if hans_eval[i]['label'] == -1 :
          label = 1
        else :
          label = hans_eval[i]['label']
        dataset_new.update( {
            'premise' : hans_eval[i]['sentence1']
            ,'hypothesis' : hans_eval[i]['sentence2']
            , 'label' : label
            })

        json.dump(dataset_new, f)
        f.write('\n')


30000


In [19]:
from datasets import Dataset
#print(hans_eval)
hans_dataset = Dataset.from_list(hans_eval)

In [20]:
print( hans_dataset)

Dataset({
    features: ['heuristic', 'template', 'sentence1_binary_parse', 'sentence2_parse', 'subcase', 'sentence2_binary_parse', 'sentence1', 'sentence2', 'sentence1_parse', 'gold_label', 'pairID', 'label'],
    num_rows: 30000
})


Fine-tune pretrained model on HANS Train set

In [22]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Step 1: Load the HANS dataset
# Convert to Hugging Face Dataset
hans_dataset = Dataset.from_list(hans_eval)

# Step 2: Load the Tokenizer
# Replace 'google/electra-base-discriminator' with your model's tokenizer if different
# Load Electra Small model and tokenizer
model_name = "/content/drive/MyDrive/electra/trained_model/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)  # 3 labels for SNLI

# Tokenize the HANS dataset
def preprocess(example):
    return tokenizer(
        example["sentence1"],
        example["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

tokenized_hans = hans_dataset.map(preprocess, batched=True)

# Step 3: Split the Dataset (Optional)
# Use 90% for training and 10% for validation
split_data = tokenized_hans.train_test_split(test_size=0.1)
train_dataset = split_data["train"]
eval_dataset = split_data["test"]

# Step 4: Load the Pretrained Model
# Replace '/path/to/your/pretrained_model' with the directory where your model is saved
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,  # Path to your pretrained model
    num_labels=3  # HANS has 2 labels: entailment and non-entailment
)

# Step 5: Define Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/electra/output_analysis/HANS/output/",  # Directory to save the fine-tuned model
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",                # Save the model at the end of each epoch
    learning_rate=5e-6,                   # Fine-tuning learning rate
    per_device_train_batch_size=16,       # Adjust based on GPU memory
    per_device_eval_batch_size=16,
    num_train_epochs=3,                   # Number of fine-tuning epochs
    weight_decay=0.01,                    # Regularization
    logging_dir="./hans_logs",                 # Directory for logs
    logging_steps=100,                    # Log every 100 steps
    save_total_limit=2,                   # Keep only the last 2 checkpoints
    load_best_model_at_end=True,          # Load the best model after training
    fp16=True                             # Enable mixed precision for faster training
)

# Step 6: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Step 7: Start Fine-tuning
trainer.train()

# Step 8: Save the Fine-tuned Model
trainer.save_model("/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model")
tokenizer.save_pretrained("/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model")


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-2b0c8d6d0f17>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.144100,0.067314
2,0.095100,0.019517
3,0.040000,0.014641


('/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model/vocab.txt',
 '/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model/added_tokens.json',
 '/content/drive/MyDrive/electra/output_analysis/HANS/model/hans_fine_tuned_model/tokenizer.json')

# Function to generate Error Classes from Eval_predictions.json

In [5]:
import numpy as np
import pandas as pd
from datasets import Dataset
import json


def load_jsonl_predictions(file_path):
      json_dataset = []  # Initialize an empty list to store JSON objects
      with open(file_path, 'r') as f:
          for line in f:  # Read the file line by line
              try:
                  data = json.loads(line)  # Parse each line as a JSON object
                  json_dataset.append(data)  # Add the parsed object to the list
              except json.JSONDecodeError as e:
                  print(f"Error decoding JSON: {e}, skipping line: {line}")

      dataset = Dataset.from_list(json_dataset)
      # Example Data (Replace with actual data)
      data = json_dataset

      num_classes = 3  # Number of label classes



      def token_overlap_ratio(premise, hypothesis):
          premise_tokens = set(premise.split())
          hypothesis_tokens = set(hypothesis.split())
          intersection = len(premise_tokens & hypothesis_tokens)
          return intersection / len(premise_tokens)

      # Define Error Classes
      error_classes = {
          "Negation": lambda x: any(word in x["premise"].split(" ") + x["hypothesis"].split(" ") for word in ["not", "no", "never", "without"]),
          "Coreference": lambda x: any(word in x["hypothesis"] for word in ["he", "she", "it", "they"]),
          "Rare Words": lambda x: any(word in x["premise"] for word in ["Higgs", "boson", "quantum"]),
          "Ambiguity": lambda x: any(word in x["hypothesis"] for word in ["something", "someone", "several"]),
          "Long Sentences": lambda x: len(x["premise"].split()) > 10 or len(x["hypothesis"].split()) > 10,
          "High Lexical Overlap": lambda x: token_overlap_ratio(x["premise"], x["hypothesis"]) > 0.7,
      }



      # Initialize Counters
      error_counts = {cls: 0 for cls in error_classes}
      total_counts = {cls: 0 for cls in error_classes}
      error_examples = {cls: [] for cls in error_classes}

      # Process Examples
      for example in data:
          true_label = example['label']
          predicted_label = example['predicted_label']
          is_error = true_label != predicted_label  # Check if the prediction is incorrect

          for cls, rule in error_classes.items():
              if rule(example):  # Check if example matches the error class
                  total_counts[cls] += 1
                  if is_error:
                      error_counts[cls] += 1
                      error_examples[cls].append(example)

      # Calculate Metrics
      error_rates = {cls: (error_counts[cls] / total_counts[cls] * 100 if total_counts[cls] > 0 else 0) for cls in error_classes}
      frequencies = {cls: (total_counts[cls] / len(data) * 100) for cls in error_classes}

      # Create Summary Table
      summary_table = pd.DataFrame({
          "Error Class": list(error_classes.keys()),
          "Frequency (%)": [frequencies[cls] for cls in error_classes],
          "Error Rate (%)": [error_rates[cls] for cls in error_classes],
          "Error Count": [error_counts[cls] for cls in error_classes],
          "Total Count": [total_counts[cls] for cls in error_classes],
      })

      # Display Results
      print(summary_table)


Model fine tuned on HANS and evaluated on HANS dataset

In [23]:
!python3 /content/drive/MyDrive/electra/fp-dataset-artifacts-main/run.py --do_eval --task nli --dataset /content/drive/MyDrive/electra/output_analysis/HANS/eval_hans_lexical_overlap.json --model /content/drive/MyDrive/electra/output_analysis/HANS/output/checkpoint-5064/ --output_dir /content/drive/MyDrive/electra/output_analysis/HANS/output/eval/

2024-11-28 01:50:46.490650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 01:50:46.510002: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 01:50:46.515838: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 01:50:47.533187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Preprocessing data... (this takes a little bit, should only happen once per dataset)
Map (num_proc=2): 100% 10000/10000 [00:01<00:00, 6739.96 examples/s]
/content/drive/MyDrive/electra/fp-dataset-artifacts-main/run.py:159: FutureWarning: `to

Testing HANS fine-tuned model on SNLI

In [3]:
!python3 /content/drive/MyDrive/electra/fp-dataset-artifacts-main/run.py --do_eval --task nli --dataset snli --model /content/drive/MyDrive/electra/output_analysis/HANS/output/checkpoint-5064/ --output_dir /content/drive/MyDrive/electra/output_analysis/HANS

2024-11-28 02:03:58.306060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 02:03:58.325660: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 02:03:58.331581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 02:03:58.345949: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 02:03:59.663388: W tensorflow/comp

Reloading Eval predictions

In [4]:
import json
json_dataset = []  # Initialize an empty list to store JSON objects
with open("/content/drive/MyDrive/electra/output_analysis/HANS/eval_predictions.jsonl", 'r') as f:
    for line in f:  # Read the file line by line
        try:
            data = json.loads(line)  # Parse each line as a JSON object
            json_dataset.append(data)  # Add the parsed object to the list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}, skipping line: {line}")

**Testing pretrained model on ambiguous top 33 percent examples**

In [3]:
!python3 /content/drive/MyDrive/electra/fp-dataset-artifacts-main/run.py --do_eval --task nli --dataset /content/drive/MyDrive/electra/cartography/datasets/ambiguous_dataset_33_percent.json --model /content/drive/MyDrive/electra/trained_model/ --output_dir /content/drive/MyDrive/electra/output_analysis/HANS

2024-11-28 03:48:20.564616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 03:48:20.640097: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 03:48:20.680674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 03:48:20.736094: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 03:48:24.255921: W tensorflow/comp

**Testing HANS Fine-tuned model on ambiguous top 33 percent examples**

In [4]:
!python3 /content/drive/MyDrive/electra/fp-dataset-artifacts-main/run.py --do_eval --task nli --dataset /content/drive/MyDrive/electra/cartography/datasets/ambiguous_dataset_33_percent.json --model /content/drive/MyDrive/electra/output_analysis/HANS/output/checkpoint-5064/ --output_dir /content/drive/MyDrive/electra/output_analysis/HANS

2024-11-28 03:50:35.152796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 03:50:35.172246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 03:50:35.178205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 03:50:35.192321: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 03:50:36.218714: W tensorflow/comp

Error Categories for Pretrained Model

In [6]:

load_jsonl_predictions("/content/drive/MyDrive/electra/output_analysis/HANS/amb33_eval_predictions.jsonl")

            Error Class  Frequency (%)  Error Rate (%)  Error Count  \
0              Negation       2.297619       13.816926           80   
1           Coreference      69.095238       19.934528         3471   
2            Rare Words       0.000000        0.000000            0   
3             Ambiguity       1.087302       18.978102           52   
4        Long Sentences      64.218254       20.126058         3257   
5  High Lexical Overlap       2.908730       22.100955          162   

   Total Count  
0          579  
1        17412  
2            0  
3          274  
4        16183  
5          733  


Error Categories for HANS Fine-tuned Model

In [7]:
load_jsonl_predictions("/content/drive/MyDrive/electra/output_analysis/HANS/eval_predictions.jsonl")

            Error Class  Frequency (%)  Error Rate (%)  Error Count  \
0              Negation       2.297619       22.107081          128   
1           Coreference      69.095238       30.335401         5282   
2            Rare Words       0.000000        0.000000            0   
3             Ambiguity       1.087302       32.846715           90   
4        Long Sentences      64.218254       30.012977         4857   
5  High Lexical Overlap       2.908730       32.878581          241   

   Total Count  
0          579  
1        17412  
2            0  
3          274  
4        16183  
5          733  


In [19]:

num_classes = 3  # Number of label classes

# Define Error Classes
error_classes = {
    "Negation": lambda x: any(word in x["premise"].split(" ") + x["hypothesis"].split(" ") for word in ["not", "no", "never", "without"]),
    "Coreference": lambda x: any(word in x["hypothesis"] for word in ["he", "she", "it", "they"]),
    "Rare Words": lambda x: any(word in x["premise"] for word in ["Higgs", "boson", "quantum"]),
    "Ambiguity": lambda x: any(word in x["hypothesis"] for word in ["something", "someone", "several"]),
    "Long Sentences": lambda x: len(x["premise"].split()) > 10 or len(x["hypothesis"].split()) > 10,
}

# Initialize Counters
error_counts = {cls: 0 for cls in error_classes}
total_counts = {cls: 0 for cls in error_classes}
error_examples = {cls: [] for cls in error_classes}

# Process Examples
for example in data:
    true_label = example['label']
    #predicted_label = example['predicted_label']
    #is_error = true_label != predicted_label  # Check if the prediction is incorrect

    for cls, rule in error_classes.items():
        if rule(example):  # Check if example matches the error class
            total_counts[cls] += 1
            error_examples[cls].append(example)

# Calculate Metrics
#error_rates = {cls: (error_counts[cls] / total_counts[cls] * 100 if total_counts[cls] > 0 else 0) for cls in error_classes}
frequencies = {cls: (total_counts[cls] / len(data) * 100) for cls in error_classes}

# Create Summary Table
summary_table = pd.DataFrame({
    "Error Class": list(error_classes.keys()),
    "Frequency (%)": [frequencies[cls] for cls in error_classes],
#    "Error Rate (%)": [error_rates[cls] for cls in error_classes],
#    "Error Count": [error_counts[cls] for cls in error_classes],
    "Total Count": [total_counts[cls] for cls in error_classes],
})

# Display Results
print(summary_table)

output_path = "/content/drive/MyDrive/electra/output_analysis/"


for k,v in error_examples.items():
  with open(output_path + k +".json", "w") as f:
    for i in v:
        json.dump(i, f, indent=4)
        f.write('\n')


      Error Class  Frequency (%)  Total Count
0        Negation       1.730976         9523
1     Coreference      64.768282       356324
2      Rare Words       0.000000            0
3       Ambiguity       1.173130         6454
4  Long Sentences      63.061481       346934


In [ ]:
threshold = 0.6  # Example threshold for high overlap (Jaccard Similarity)
for example in data:
    premise = example["premise"]
    hypothesis = example["hypothesis"]


    overlap_ratio = token_overlap_ratio(premise, hypothesis)

    # Determine if high lexical overlap
    if  overlap_ratio > threshold:
        category = "High Lexical Overlap"
    else:
        category = "Low Lexical Overlap"


In [5]:
import numpy as np
import pandas as pd
from datasets import Dataset

dataset = Dataset.from_list(json_dataset)
# Example Data (Replace with actual data)
data = json_dataset

num_classes = 3  # Number of label classes



def token_overlap_ratio(premise, hypothesis):
    premise_tokens = set(premise.split())
    hypothesis_tokens = set(hypothesis.split())
    intersection = len(premise_tokens & hypothesis_tokens)
    return intersection / len(premise_tokens)

# Define Error Classes
error_classes = {
    "Negation": lambda x: any(word in x["premise"].split(" ") + x["hypothesis"].split(" ") for word in ["not", "no", "never", "without"]),
    "Coreference": lambda x: any(word in x["hypothesis"] for word in ["he", "she", "it", "they"]),
    "Rare Words": lambda x: any(word in x["premise"] for word in ["Higgs", "boson", "quantum"]),
    "Ambiguity": lambda x: any(word in x["hypothesis"] for word in ["something", "someone", "several"]),
    "Long Sentences": lambda x: len(x["premise"].split()) > 10 or len(x["hypothesis"].split()) > 10,
    "High Lexical Overlap": lambda x: token_overlap_ratio(x["premise"], x["hypothesis"]) > 0.7,
}



# Initialize Counters
error_counts = {cls: 0 for cls in error_classes}
total_counts = {cls: 0 for cls in error_classes}
error_examples = {cls: [] for cls in error_classes}

# Process Examples
for example in data:
    true_label = example['label']
    predicted_label = example['predicted_label']
    is_error = true_label != predicted_label  # Check if the prediction is incorrect

    for cls, rule in error_classes.items():
        if rule(example):  # Check if example matches the error class
            total_counts[cls] += 1
            if is_error:
                error_counts[cls] += 1
                error_examples[cls].append(example)

# Calculate Metrics
error_rates = {cls: (error_counts[cls] / total_counts[cls] * 100 if total_counts[cls] > 0 else 0) for cls in error_classes}
frequencies = {cls: (total_counts[cls] / len(data) * 100) for cls in error_classes}

# Create Summary Table
summary_table = pd.DataFrame({
    "Error Class": list(error_classes.keys()),
    "Frequency (%)": [frequencies[cls] for cls in error_classes],
    "Error Rate (%)": [error_rates[cls] for cls in error_classes],
    "Error Count": [error_counts[cls] for cls in error_classes],
    "Total Count": [total_counts[cls] for cls in error_classes],
})

# Display Results
print(summary_table)


            Error Class  Frequency (%)  Error Rate (%)  Error Count  \
0              Negation       1.574883       16.129032           25   
1           Coreference      64.661654       13.513514          860   
2            Rare Words       0.000000        0.000000            0   
3             Ambiguity       1.158301       18.421053           21   
4        Long Sentences      68.258484       12.473951          838   
5  High Lexical Overlap       3.881325        9.424084           36   

   Total Count  
0          155  
1         6364  
2            0  
3          114  
4         6718  
5          382  
